# Lorenz

In this notebook we are going to syncronize a subsystem of a Lorenz system with another Lorenz system according to the proposed method of Pecora and Carroll (1990).

In [2]:
from plotly import offline as py
from plotly import graph_objs as go

py.init_notebook_mode(connected=True)

A first system governed by the Lorenz,
$$
\begin{align}
\dot{x}_t&=\sigma(y_t-x_t)\\
\dot{y}_t&=x_t(r-z_t)-y_t\\
\dot{z}_t&=x_ty_t-bz_t
\end{align},
$$
acts as a transmitter with $r=60,\sigma=10,b=8/3$.

A second system governed by,
$$
\begin{align}
\dot{x}_r&=x_d\\
\dot{y}_r&=x_r(r-z_r)-y_r\\
\dot{z}_r&=x_ry_r-bz_r
\end{align},
$$
acts as a receiver, using the $x_d$ from the transmitter.

In [44]:
from numba import jit
from random import uniform

r = 60
b = 8 / 3
sigma = 10


@jit(nopython=True)
def simulate(N, h):
    xt = [uniform(0, 10)]
    yt = [uniform(-60, 60)]
    zt = [uniform(0, 140)]
    
    xr = xt
    yr = [uniform(-60, 60)]
    zr = [uniform(0, 140)]
    
    for i in range(1, N+1):
        xti = xt[-1]
        yti = yt[-1]
        zti = zt[-1]
        
        xt.append(xti + h * sigma * (yti - xti))
        yt.append(yti + h * (xti * (r - zti) - yti))
        zt.append(zti + h * (xti * yti - b * zti))
        
        yri = yr[-1]
        zri = zr[-1]
        
        yr.append(yri + h * (xti * (r - zri) - yri))
        zr.append(zri + h * (xti * yri - b * zri))
        
    return (xt, yt, zt), (xr, yr, zr)


Introducing the error measure $e_2=y_t-y_r,e_3=z_t-z_r$ and subtracting the receiver equations from the transmitter equations one finds,
$$
\begin{align}
\dot{e}_1&=0\\
\dot{e}_2&=-e_2-x_de_3\\
\dot{e}_3&=x_de_2-be_3
\end{align}.
$$
If we multiply the second equation by $e_2$, the third equation by $4e_3$ and add the results we obtain,
$$
-2\left(e_2^2+e_3^2\right)
=
e_2\dot{e}_2+e_3\dot{e}_3
=
\frac{1}{2}\frac{d}{dt}\left(e_2^2+e_3^2\right),
$$
we identify $V(e_2,e_3)=e_2^2+e_3^2$ and see that $V$ is a Lyapunov function as $V$ it is positive definite and $\dot{V}$ is negative definite, thus, $e_2=e_3=0$ is a stable equilibrium point which one approaches asymptopically for $t\to\infty$ regardless of the initial conditions chosen for transmitter and receiver system, thus, the receiver will syncronize with the transmitter.

In [45]:
N = 1000
h = 1e-2

t = np.arange(N) * h

(xt, yt, zt), (xr, yr, zr) = simulate(1000, 1e-2)

(xt[0], xr[0]), (yt[0], yr[0]), (zt[0], zr[0])

((6.324940084560655, 6.324940084560655),
 (35.15227077484603, -16.044206306661586),
 (127.13535435788735, 126.39523401898253))

In [46]:
layout = go.Layout(
    xaxis=dict(title='t'),
    yaxis=dict(title='y(t)'),
)

figure = go.Figure([
    go.Scatter(x=t, y=yt, mode='lines+markers', name='transmitter'),
    go.Scatter(x=t, y=yr, mode='lines+markers', name='receiver'),
], layout)

py.iplot(figure)

In [47]:
layout = go.Layout(
    xaxis=dict(title='y(t)'),
    yaxis=dict(title='z(t)'),
)

figure = go.Figure([
    go.Scatter(x=yt, y=zt, mode='lines+markers', name='transmitter'),
    go.Scatter(x=yr, y=zr, mode='lines+markers', name='receiver'),
], layout)

py.iplot(figure)